# Shakespeare's Principal Components
## Shakespeare and His Plays
Shakespeare's work is ideal for statistical analysis. He wrote many plays that vary widely from each other but he returns to the same themes in each one, sometimes even with the same characters. Still, his earliest audiences and readers recognized that some plays were similar to each other and others different. The First Folio, which was the first major printing to combine most of his works, grouped the plays in three categories: Comedies, Histories and Tragedies.

| Comedies | Histories | Tragedies |
|----------|-----------|-----------|
|All's Well That Ends Well |Henry IV, Part I|Antony and Cleopatra|
|As You Like It |Henry IV, Part II|Cymbeline|
|Comedy of Errors |Henry V|Hamlet|
|Love's Labour's Lost |Henry VI, Part I|Julius Caesar|
|Measure for Measure |Henry VI, Part II|Coriolanus|
|Merchant of Venice |Henry VI, Part III|King Lear|
|Merry Wives of Windsor |King John|Macbeth|
|Midsummer Night's Dream |Pericles|Othello|
|Much Ado about Nothing|Richard II|Romeo and Juliet|
|Taming of the Shrew|Richard III|Timon of Athens|
|Tempest||Titus Andronicus|
|Twelfth Night||Troilus and Cressida|
|Two Gentlemen of Verona|||
|Two Noble Kinsmen|||
|Winter's Tale|||

## Text Analysis


## Principal Component Analysis (PCA)
PCA creates combinations of the variables in data that represent most of the variance in the data. In particular, it is helpful to reduce a set of many variables into the key variables or combinations of variables that is most responsible for changes in the data. 


## Preparing the Data
Folger Digital Texts provides a stripped version of the plays.

## Analyzing Language Action Types
Docuscope is a third-party tool that decomposes texts into Language Action Types (LATs). 

## Loading Data 
I use python to load the prepared files.

In [ ]:
import pandas as pd

file_path = "data/CSV/ubiquity-2015-06-23-06-38-28.csv"
df = pd.read_csv(file_path)